In [3]:
import pandas as pd
from math import atan2, degrees

In [4]:
data = pd.read_parquet("G:/My Drive/שנה ב/פרויקט מעבדה/front_head_only_resnet_152__top_20240103T103519.parquet")
data = data.iloc[:,0:13]

In [5]:
def calculate_direction(x_ears, y_ears, x_nose, y_nose):
    delta_x = x_ears - x_nose
    delta_y = y_ears - y_nose
    
    # Calculate direction angle using arctangent
    angle_rad = atan2(delta_y, delta_x)

    # Convert angle from radians to degrees
    direction_angle = degrees(angle_rad)

    # Normalize the angle to be in the range [0, 360)
    direction_angle = (direction_angle + 360) % 360

    return direction_angle

In [6]:
azimuth_angle_lst = []
for i in range(len(data)):
    x_ears = data['mid_ears', 'cam_x'][i]
    y_ears = data['mid_ears', 'cam_y'][i]
    x_nose = data['nose', 'cam_x'][i]
    y_nose = data['nose', 'cam_y'][i]
    azimuth_angle_lst.append(calculate_direction(x_ears, y_ears, x_nose, y_nose))

data['azimuth_angle'] = azimuth_angle_lst

In [7]:
data

left_ear                          mid_ears                        \
            cam_x       cam_y      prob       cam_x       cam_y      prob   
0      217.995560  603.225281  0.996496  224.177246  573.029297  0.996017   
1      218.083710  603.354797  0.996787  224.355301  573.049927  0.995957   
2      218.388214  603.366150  0.996721  224.385757  573.124573  0.995933   
3      218.343658  603.411804  0.996879  224.449982  573.111328  0.995997   
4      218.272507  603.337585  0.997007  224.335236  573.111206  0.996428   
...           ...         ...       ...         ...         ...       ...   
18406  208.320984  554.276733  0.972703  207.853989  525.996643  0.977414   
18407  208.356583  554.148132  0.972446  207.889740  525.923889  0.977019   
18408  208.342865  554.177246  0.970373  207.901688  525.953918  0.976052   
18409  208.409058  554.175537  0.969821  207.913193  525.947510  0.975960   
18410  201.142242  551.486572  0.998919  207.049255  523.416809  0.998759   

              nose                          right_ear                        \
             cam_x        cam_y      prob       cam_x       cam_y      prob   
0      1193.508667  1012.812195  0.969148  230.358917  542.833313  0.995537   
1      1193.516113  1012.767029  0.971888  230.626892  542.745117  0.995126   
2      1193.454834  1012.724182  0.971763  230.383301  542.882996  0.995146   
3      1193.522949  1012.768677  0.972229  230.556290  542.810913  0.995114   
4      1193.549683  1012.830017  0.971937  230.397964  542.884888  0.995849   
...            ...          ...       ...         ...         ...       ...   
18406  1193.485352  1012.838928  0.927187  207.386993  497.716583  0.982125   
18407  1193.500122  1012.846130  0.929073  207.422882  497.699677  0.981591   
18408  1193.480591  1012.841858  0.928149  207.460510  497.730560  0.981731   
18409  1193.489746  1012.842041  0.928060  207.417328  497.719543  0.982099   
18410  1193.592529  1012.878052  0.928386  212.956253  495.347046  0.998599   

               time azimuth_angle  
                                   
0      1.704278e+09    204.403660  
1      1.704278e+09    204.404229  
2      1.704278e+09    204.400509  
3      1.704278e+09    204.403253  
4      1.704278e+09    204.403119  
...             ...           ...  
18406  1.704278e+09    206.286587  
18407  1.704278e+09    206.290808  
18408  1.704278e+09    206.289931  
18409  1.704278e+09    206.290294  
18410  1.704278e+09    206.387740  

[18411 rows x 14 columns]